# LanceDB fundamentals

LanceDB is an open-source vector database designed to handle large-scale vector data efficiently. It provides a robust platform for storing, indexing, and querying high-dimensional vectors, making it ideal for applications in machine learning, computer vision, and natural language processing.

We'll be using LanceDB for the purpose of creating RAG applications. However before we dive into that, let's cover some fundamental concepts of LanceDB, so that you know what is happening under the hood.

## Connect LanceDB

- when connecting to LanceDB it will create a folder for your lancedb database if it does not already exist

In [1]:
import lancedb

db = lancedb.connect(uri = "vector_database")
db

LanceDBConnection(uri='/Users/aigineer/Documents/explorations/AI_engineering_learning/lancedb_vector_database/lancedb_lecture/vector_database')

In [2]:
db.uri

'/Users/aigineer/Documents/explorations/AI_engineering_learning/lancedb_vector_database/lancedb_lecture/vector_database'

## Create a Table

when creating a table, it is stored as a folder with .lance suffix containing the following folders
- _transactions
- _versions
- data

a table is a pyarraow table stored in the data folder

In [3]:
import json
with open("animals_text_embeddings.json") as file:
    data = json.loads(file.read())

data

[{'text': 'A small brown dog running.', 'vector': [0.12, 0.85, 0.33]},
 {'text': 'A cat resting quietly on a sofa.', 'vector': [0.4, 0.91, 0.1]},
 {'text': 'A large gray elephant drinking water.',
  'vector': [0.88, 0.22, 0.55]},
 {'text': 'A fast cheetah sprinting across the savannah.',
  'vector': [0.95, 0.12, 0.72]},
 {'text': 'A colorful parrot perched on a branch.',
  'vector': [0.25, 0.66, 0.81]},
 {'text': 'A frog sitting on a lily pad.', 'vector': [0.14, 0.44, 0.27]}]

In [4]:
table_animals = db.create_table("animals_text", exist_ok=True, data=data)
table_animals


LanceTable(name='animals_text', version=2, _conn=LanceDBConnection(uri='/Users/aigineer/Documents/explorations/AI_engineering_learning/lancedb_vector_database/lancedb_lecture/vector_database'))

In [5]:
# it's a pyarrow table 
table_animals.head()

pyarrow.Table
text: string
vector: fixed_size_list<item: float>[3]
  child 0, item: float
----
text: [["A small brown dog running.","A cat resting quietly on a sofa.","A large gray elephant drinking water.","A fast cheetah sprinting across the savannah.","A colorful parrot perched on a branch."]]
vector: [[[0.12,0.85,0.33],[0.4,0.91,0.1],[0.88,0.22,0.55],[0.95,0.12,0.72],[0.25,0.66,0.81]]]

In [6]:
# converting to pandas 
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
7,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"


add some more data to the table

In [7]:
more_data = [
    {"text": "A panda eating bamboo peacefully.", "vector": [0.51, 0.37, 0.82]},
    {"text": "A lion roaring loudly on a rock.", "vector": [0.93, 0.18, 0.41]},
]

table_animals.add(more_data)

AddResult(version=3)

In [8]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
7,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"
8,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
9,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"


## Create an empty table and then delete it

In [9]:
# LanceModel is a Pydantic model that can be converted to LanceDB table
from lancedb.pydantic import LanceModel

class JokeSchema(LanceModel):
    joke: str 
    rating: int


db.create_table(name = "jokes",schema=JokeSchema, exist_ok=True)

LanceTable(name='jokes', version=1, _conn=LanceDBConnection(uri='/Users/aigineer/Documents/explorations/AI_engineering_learning/lancedb_vector_database/lancedb_lecture/vector_database'))

In [10]:
db.table_names()

['animals_text', 'jokes']

In [11]:
db.drop_table("jokes")

In [12]:
db.table_names()

['animals_text']

## Open existing table

In [13]:
# since db is already connected to vector_database it has animals_text as table
db.open_table("animals_text").head()

pyarrow.Table
text: string
vector: fixed_size_list<item: float>[3]
  child 0, item: float
----
text: [["A small brown dog running.","A cat resting quietly on a sofa.","A large gray elephant drinking water.","A fast cheetah sprinting across the savannah.","A colorful parrot perched on a branch."]]
vector: [[[0.12,0.85,0.33],[0.4,0.91,0.1],[0.88,0.22,0.55],[0.95,0.12,0.72],[0.25,0.66,0.81]]]

## Vector search in LanceDB

Uses ANN - approximate nearest neighbours for searching vectors

- search with vector directly
- note that the vector column is already an example embedding simulated with LLM

A more realistic approach is to just have text records, use an existing embedding model and embedd those text records, then embed the query vector and find closest match 


In [14]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
7,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"
8,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
9,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"


In [ ]:
query_vector = [0.5, 0.2, 0.9]

# finds closest rows based on the Euclidean distance by default
table_animals.search(query_vector).limit(3).to_pandas()

,text,vector,_distance
0,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]",0.0354
1,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]",0.0354
2,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]",0.2413


## Embeddings API

LanceDB provides an Embeddings API that allows you to easily generate and store vector embeddings for your data. This is particularly useful for applications such as semantic search, recommendation systems, and clustering.

We'll use embeddings model from Gemini but, you can use any embedding model of your choice.

Embeddings are vectors that represent the semantic meaning of data, such as text or images. By converting data into embeddings, you can perform similarity searches and other operations more effectively

In [16]:
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

model = get_registry().get("gemini-text").create(name = "gemini-embedding-001")

class JokeModel(LanceModel):
    joke: str = model.SourceField() # marks text as input to embedding function
    vector: Vector(3072) = model.VectorField() # vector is destination of computed embedding

table_jokes = db.create_table("jokes", schema = JokeModel, exist_ok=True)
table_jokes



LanceTable(name='jokes', version=1, _conn=LanceDBConnection(uri='/Users/aigineer/Documents/explorations/AI_engineering_learning/lancedb_vector_database/lancedb_lecture/vector_database'))

In [27]:
import pandas as pd 
with open("jokes.json", "r") as file:
    jokes_data = json.loads(file.read())

df_jokes = pd.DataFrame(jokes_data).rename({"jokes": "joke"}, axis=1)
df_jokes.head()

,joke
0,Parallel lines have so much in common—it’s sad...
1,"ETL stands for “Extract, Transform, Leave for ..."
2,What do you call a snake that runs your script...
3,"Gold walks into a bar. The bartender says, “Au..."
4,C# devs don’t argue; they just throw exceptions.


adds data to our table

In [28]:
table_jokes.add(df_jokes)


/Users/aigineer/Documents/explorations/AI_engineering_learning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AddResult(version=2)

we can see that vector column has been created automatically and embeddings have been generated for each joke

In [30]:
table_jokes.head(2)

pyarrow.Table
joke: string not null
vector: fixed_size_list<item: float>[3072]
  child 0, item: float
----
joke: [["Parallel lines have so much in common—it’s sad they’ll never meet.","ETL stands for “Extract, Transform, Leave for the next person.”"]]
vector: [[[-0.024001757,0.01247358,-0.024144737,-0.06704516,0.017059995,...,0.011402721,-0.017770408,0.011606138,0.0004488597,0.012175956],[-0.015356114,0.0211365,-0.021389864,-0.07957475,0.008829045,...,0.023671096,0.00070166244,0.003873497,0.0006493248,0.01587487]]]

In [31]:
table_jokes.to_pandas().head()

,joke,vector
0,Parallel lines have so much in common—it’s sad...,"[-0.024001757, 0.01247358, -0.024144737, -0.06..."
1,"ETL stands for “Extract, Transform, Leave for ...","[-0.015356114, 0.0211365, -0.021389864, -0.079..."
2,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0..."
3,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0..."
4,C# devs don’t argue; they just throw exceptions.,"[-0.0068662865, -0.005415149, 0.0044965413, -0..."


In [32]:
table_jokes.search("data engineering jokes").limit(8).to_pandas()

,joke,vector,_distance
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.461179
1,"Data engineer motto: If it works, don’t touch ...","[-0.020296954, 0.020327171, -0.009069326, -0.0...",0.542293
2,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.655434
3,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.667807
4,I tried to explain async/await to my friend… n...,"[-0.016174542, 0.012602222, -0.015254588, -0.0...",0.679036
5,I asked the data lake if it had my file. It sa...,"[-0.026169129, 0.01796462, -0.013938847, -0.09...",0.680632
6,Why did the Python programmer get bitten? Beca...,"[-0.021944793, 0.0030636177, -0.019837778, -0....",0.684943
7,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.701690


In [34]:
table_jokes.search("give me jokes on C#").limit(8).to_pandas()

,joke,vector,_distance
0,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.458438
1,Why did the C# developer go broke? He kept usi...,"[-0.02175711, 0.0073709465, -0.015515881, -0.0...",0.483707
2,Why is C# like a musical? It has so many classes.,"[-0.012926053, 0.0034431089, -0.017093537, -0....",0.524282
3,C# devs don’t argue; they just throw exceptions.,"[-0.0068662865, -0.005415149, 0.0044965413, -0...",0.614111
4,Why did the Python programmer get bitten? Beca...,"[-0.021944793, 0.0030636177, -0.019837778, -0....",0.627615
5,C# programmers love coffee—they're always work...,"[-0.0079965545, 0.0151333995, -0.009256328, -0...",0.630019
6,I tried to explain async/await to my friend… n...,"[-0.016174542, 0.012602222, -0.015254588, -0.0...",0.632899
7,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.644982


In [43]:
table_jokes.search("give me chemistry related jokes").limit(8).to_pandas()

,joke,vector,_distance
0,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.576598
1,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.613355
2,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.671744
3,Chemists have all the solutions… mostly in bea...,"[-0.012411982, -0.002532549, -0.0061755716, -0...",0.673519
4,Oxygen and Magnesium got together—OMg!,"[-0.024720555, 0.006975079, -0.015012385, -0.0...",0.738241
5,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.762712
6,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.765103
7,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.773378


In [44]:
table_jokes.search("give me nature related jokes").limit(8).to_pandas()

,joke,vector,_distance
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.592044
1,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.693084
2,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.723715
3,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.725794
4,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.750098
5,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.761704
6,Why did the Python programmer get bitten? Beca...,"[-0.021944793, 0.0030636177, -0.019837778, -0....",0.770420
7,Why’s 6 afraid of 7? Because 7 8 9.,"[-0.040820926, 0.0074244644, -0.02972158, -0.0...",0.775439


## Hybrid search

hybrid search combines traditional keyword-based search with vector-based search to provide more relevant results. This approach leverages the strengths of both methods, allowing for more accurate and context-aware retrieval of information.

In [39]:
# to enable keyword based search on joke column
table_jokes.create_fts_index("joke", replace=True)


In [ ]:
from lancedb import rerankers

# reranks the results from a vector and full text search
reranker = rerankers.RRFReranker()

results = table_jokes.search(
    "give me nature related jokes",
    query_type="hybrid",
    vector_column_name="vector",
    fts_columns="joke",
).rerank(reranker).limit(8).to_pandas()

# somewhat different to before
results


,joke,vector,_relevance_score
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.032522
1,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.032018
2,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.016129
3,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.015873
4,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.015385
5,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.015152
6,Why did the Python programmer get bitten? Beca...,"[-0.021944793, 0.0030636177, -0.019837778, -0....",0.014925
7,Why’s 6 afraid of 7? Because 7 8 9.,"[-0.040820926, 0.0074244644, -0.02972158, -0.0...",0.014706


## Rule of thumb 

- For exact matching -> FTS
- For meaning based matching -> Vector search
- Unpredictable or mixed queries -> Hybrid search
